In [1]:
import pandas as pd, plotly.express as px, plotly.graph_objects as go
import numpy as np
import math
import glob
import scipy.signal as signal, scipy.ndimage as ndimage
from pathlib import Path

In [2]:
def smooth_savgol_octave(freq, spl, fraction=1/24):
    """
    Smoothing using Savitzky-Golay filter in log frequency domain
    """
    # Sort by frequency (important for proper filtering)
    sort_idx = np.argsort(freq)
    freq_sorted = freq[sort_idx]
    spl_sorted = spl[sort_idx]
    
    # Convert to log frequency
    log_freq = np.log10(freq_sorted)
    
    # Calculate window length based on data density and octave fraction
    log_freq_range = log_freq.max() - log_freq.min()
    points_per_octave = len(log_freq) / log_freq_range
    window_points = max(5, int(points_per_octave * fraction))
    if window_points % 2 == 0:
        window_points += 1
    
    # Apply Savitzky-Golay filter
    smoothed_spl = signal.savgol_filter(spl_sorted, window_points, 3)
    
    # Return original order
    result = np.zeros_like(spl_sorted)
    result[sort_idx] = smoothed_spl
    
    return freq, result

In [3]:
colors = px.colors.qualitative.Set1

In [5]:
icolor = 0
fig = {}
for i, csv_file in enumerate(sorted(glob.glob("*.csv"))):
    try:
        channel = csv_file[10:11]
        if channel not in fig:
            icolor = 0
            fig[channel] = go.Figure()
        else:
            icolor += 1
        df = pd.read_csv(csv_file)
        if 'frequency_hz' not in df.columns or 'spl_db' not in df.columns:
            continue
        # Extract and clean data
        freq = df['frequency_hz'].values
        spl = df['spl_db'].values
        phase = df['phase_deg'].values
        valid = np.isfinite(freq) & np.isfinite(spl) & np.isfinite(phase)
        freq, spl = freq[valid], spl[valid]
        # Apply smoothing
        octave = 24
        smoothed_freq, smoothed_spl = freq, spl # smooth_savgol_octave(freq, spl, 1.0/octave)
        # Create nice name
        name = Path(csv_file).stem.replace('_', ' ').title()
        # Add to plot
        fig[channel].add_trace(go.Scatter(
            x=freq, 
            y=spl, 
            name='{} (raw)'.format(name), 
            line=dict(color=colors[icolor % len(colors)], width=2.5)
        ))
        icolor += 1
        fig[channel].add_trace(go.Scatter(
            x=smoothed_freq, 
            y=smoothed_spl, 
            name='{} (smoothed 1/{})'.format(name, octave), 
            line=dict(color=colors[icolor % len(colors)], width=2.5)
        ))

    except Exception as e:
        print(f"✗ {csv_file}: {e}")

for f in fig.values():   
    min_spl = 1000
    max_spl = -1000
    for t in f.data:
        min_spl = min(min_spl, np.min(t.y))
        max_spl = max(max_spl, np.max(t.y))
    max_spl = 5*(int(max_spl) // 5)+5
    min_spl = 5*(int(min_spl) // 5)-5
    f.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    f.update_layout(
      title='Frequency Response - 1/24 Octave Smoothed SPL',
      xaxis_title='Frequency (Hz)',
      yaxis_title='Sound Pressure Level (dB)',
      yaxis_range=[min_spl, max_spl],
      xaxis_type='log',
      xaxis_range=[1+math.log10(2), 4+math.log10(2)],
      height=450,
      showlegend=True,
      plot_bgcolor='white'
    )
    f.show()
    

ValueError: zero-size array to reduction operation minimum which has no identity